# Байесовская оценка А/Б-тестов

*Сформулированы критерии оценки А/Б-тестов. Рассмотрены примеры байесовского моделирования. Байесовская оценка применена к сравнению конверсий, средних с помощью центральной предельной теоремы, выручки на пользователя, заказов на платящего, средних чеков.*

&nbsp; &nbsp; *- [А/Б тесты](#А/Б-тесты)*  
&nbsp; &nbsp; *- [Байесовское моделирование](#Байесовское-моделирование)*  
&nbsp; &nbsp; *- [Конверсии](#Конверсии)*   
&nbsp; &nbsp; *- [Средние](#Средние)*    
&nbsp; &nbsp; *- [Выручка на пользователя](#Выручка-на-пользователя)*  
&nbsp; &nbsp; *- [Заказы на платящего](#Заказы-на-платящего)*  
&nbsp; &nbsp; *- [Средний чек](#Средний-чек)*  
&nbsp; &nbsp; *- [Заключение](#Заключение)*  
&nbsp; &nbsp; *- [Ссылки](#Ссылки)*  

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats

from collections import namedtuple

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

np.random.seed(7)

# А/Б тесты  

В мобильные приложения и веб-сервисы вносят изменения для улучшения ключевых метрик - выручки, конверсий, вовлеченности и др. Например, при повышении цен конверсия в покупку скорее всего снизится, но итоговая прибыль может вырасти.  

<center>
<img src="./figs/experiment_versions_ru.png" alt="experiment_versions" width="400"/>
</center>

Точный эффект от изменений непредсказуем. Новая функциональность может ухудшать продукт. По оценкам, только около трети реализованных изменений приводят к положительным результатам [[MicroExp](https://www.microsoft.com/en-us/research/publication/online-experimentation-at-microsoft/)]. Поэтому необходимо измерять эффект от новой функциональности.

После запуска эффект не всегда может быть виден сразу (см. рисунок ниже). Если функциональность не вызвала резких изменений, ее влияние может быть незаметно на фоне случайных колебаний метрик. Кроме того, могут произойти изменения в других частях продукта, привлекаемом трафике или общей активности аудитории, которые также повлияют на целевую метрику. Например, запуск рекламной акции. Поэтому изменения метрик после релиза не всегда можно объяснять именно новой функциональностью.

<center>
<img src="./figs/effect_size.png" alt="effect_size"  width="900"/>
<em>Возможный эффект от релиза. Резкие изменения могут быть заметны сразу. Падение на 30%. Слабые изменения могуть быть незаметны на фоне колебаний метрик. Улучшение на 3% эффект неочевиден.</em>
</center>

Одним из способов оценки новой функциональности является проведение А/Б-теста.
В этом подходе запускают версию без изменений и измененные варианты сервиса параллельно, распределяют пользователей между этими вариантами и сравнивают интересующие метрики. Параллельный запуск версий позволяет снизить различия из-за факторов, не связанных с тестируемыми изменениями.

В проведении АБ-тестов есть много нюансов [[TrustworthyAB]((https://www.amazon.com/gp/product/B0845Y3DJV)].
Но общая схема примерно следующая (см. рисунок). При попадании на сайт или в приложение пользователь случайным образом определяется в одну из экспериментальных групп. В каждой группе собираются данные и вычисляются интересующие метрики.
Полученные значения сравниваются между собой. Эксперимент прекращается, если становится понятно, что одна из групп лидирует (иногда - при прохождении определенного количества пользователей, по истечении определенного времени или нецелесообразности дальнейшего проведения). Принимается решение о дальнейших действиях - как правило, о выборе одного из вариантов для всех пользователей.

<center>
<img src="./figs/ab_test.png" alt="ab_test" width="800"/>
</center>

Причинная диаграмма [[CausalDAG](https://en.wikipedia.org/wiki/Causal_graph)].  
Метрики определяются действиями пользователей в приложении. Действия зависят от функциональности приложения (например, какие тарифные планы доступны). Действия разных сегментов пользователей будут отличаться (новые, ранее покупавшие). Также действия будут зависеть от внешних факторов (например, сезонность).  

В А/Б тесте версии запускают одновременно и случайно делят пользователей между версиями. При одновременном запуске влияние внешних факторов можно считать одинаковым. Влияние внешних факторов остается, но при сравнении данных из обеих групп за одинаковый период можно считать, что влияение на обе группы одинаково. Также пользователи делятся между группами случайно, можно считать состав сегментов одинаковым. В итоге разница метрик между группами объясняется разницей функциональности приложения.  

<center>
<img src="./figs/causal.png" alt="causal" width="600"/>
</center>

По итогам эксперимента нужно выбрать "лучшую" группу и оценить эффект. Для этого понадобится оценить целевые метрики в каждой группе. Точные значения метрик в каждой группе остаются неизвестыми. Эти оценки - случайные величины. Поэтому нужно оценить распределения возможных значений метрик. По мере увеличения количества данных - т.е. по мере роста количества пользователей, принявших участие в эксперименте - неопределенность в оценках снижается. Еще один вопрос - сколько должен продолжаться эксперимент?

<div style="font-size: 20px">  

Вопросы А/Б-тестов:

<ul>
<li>Какая группа лучше и насколько?</li> 
<li>Сколько должен продолжаться эксперимент?</li>
</ul>

</div>

Например, в имеющейся выборке среднее значение метрики в группе A равно a_mean, а в группе B - b_mean. При выборе варианта A для всех пользователей среднее значение метрики с вероятностью 90% будет находится в диапазоне (a_min, a_max), при выборе варианта B - с вероятностью 90% в диапазоне (b_min, b_max). По имеющимся данным группа B окажется лучше группы A с вероятностью 70%. Ожидаемое значение B/A = 1.05. Чтобы говорить о p(B) > p(A) с уверенностью 90% нужно еще около N пользователей, что займет d дней.   

Вероятность здесь и далее понимается в субъективном смысле - как мера уверенности в определенном исходе процесса с несколькими возможными исходами [[SubjProb](https://en.wikipedia.org/wiki/Probability_interpretations#Subjectivism), [UU](https://www.amazon.co.uk/Understanding-Uncertainty-Wiley-Probability-Statistics-ebook/dp/B00GYVM33Q)].

Картина А/Б тестов.

Значения метрик в группах определяются функциональностью приложения.  
Точные значения неизвестны.  
На основе собранных в эксперименте данных можно строить оценки точных значений.  
Пока данных мало, неопределенность в оценках большая.  
По мере набора данных точность увеличивается.  
С увеличением точности оценок растет уверенность, какая из групп лучше.  
Когда уверенность достигает достаточного значения, эксперимент можно останавливать.  

Байесовское моделирование позволяет оценить точное значение метрики по сэмлу и вероятность метрики в одной группе больше другой P(B>A).

<center>
<img src="./figs/ab_dynamics.png" alt="ab_dynamics" width="400"/>
</center>

# Байесовское моделирование

> Вам подарили подарок. Подарок в упаковке, и не ясно, что это. 
До того, как его развернуть, вы пытаетесь угадать, что внутри.

Эти рассуждения можно сделать количественными.  
Нужно предположить несколько вариантов подарка и для каждого оценить вероятность $P(подарок | упаковка)$. 
Это делается с помощью соотношения Байеса.

$$
P(B | A) = \frac{P(A | B) P(B)}{P(A)}
$$

В примере

$$
P(подарок_i | упаковка) = \frac{P(упаковка|подарок_i) P(подарок_i)}{P(упаковка)} = \frac{P(упаковка|подарок_i) P(подарок_i)}{\sum_i P(упаковка|подарок_i) P(подарок_i)}
$$


На этом примере видны основные этапы байесовского моделирования.  
Есть данные. Есть гипотезы, объясняющие данные.  
Для выбора одной из гипотез нужно посчитать вероятности получить данные в рамках гипотез - правдоподобия.  
Также нужны априорные вероятности каждой гипотезы.  
При комбинировании априорной вероятности гипотезы и вероятности получить данные в рамках гипотезы можно получить вероятность гипотезы при условии данных.  

> На страницу зашло 1000 человек, 100 из них нажали кнопку "Продолжить". Как выглядит распределение возможных значений конверсии? Вероятность конверсии каждого пользователя можно считать одинаковой, все возможные априорные значения конверсии равновероятными. 

Вероятность получить данные
$$
P(100/1000; \theta) = Binom(100/1000; \theta) = C^{100}_{1000} \theta^{100} (1 - \theta)^{1000-100}
$$

Плотность вероятности конверсий
$$
P(\theta; 100/1000) 
= \frac{P(100/1000; \theta) P(\theta)}{P(100/1000)}
= \frac{P(100/1000; \theta) P(\theta)}{\int d \theta P(100/1000; \theta) P(\theta)}
$$

$P(\theta)$ равномерно $P(\theta) = \mbox{Unif}(0, 1) = 1$

$$
P(\theta; 100/1000) 
= \frac{\theta^{100} (1 - \theta)^{900}}{\int d \theta (1 - \theta)^{900} \theta^{100} }
\sim \theta^{100} (1 - \theta)^{900}
= Beta(101, 901)
$$

График 

In [ ]:
ns = 100
ntotal = 1000

p_samp = ns / ntotal
p_dist = stats.beta(a=ns+1, b=ntotal-ns+1)

x = np.linspace(0, 1, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=p_dist.pdf(x), line_color='black', name='p dist'))
fig.add_trace(go.Scatter(x=[p_samp, p_samp], y=[0, max(p_dist.pdf(x))], 
                         line_color='black', mode='lines', line_dash='dash', name='p sample'))
fig.update_layout(title='Posterior Dist',
                  xaxis_title='p',
                  yaxis_title='Prob Density',
                  xaxis_range=[0, 1],
                  hovermode="x",
                  height=500)
fig.show()

> На версию А страницы веб-сайта зашло 1000 человек, 100 нажали кнопку "Продолжить". На версию Б зашло 1000 человек, 110 нажали кнопку продолжить. С какой вероятностью конверсия страницы Б выше страницы А?

Для расчета $P(model|data)$ используется связь с $P(data|model)$ - вероятностью наблюдения данных $data$ в рамках выбранной модели $model$. Связь выражается соотношением Байеса:

$$
P(model | data) = \frac{ P(data | model) P(model) }{P(data)} .
$$

Используется следующая терминология:  
$P(model | data)$ - апостериорное распределение вероятности,  
$P(data | model)$ - функция правдоподобия,  
$P(model)$ - априорное распределение вероятности,  
$P(data)$ не имеет специального названия.  

Обычно форму модели фиксируют. Меняют только параметры.  

Последовательность действий при использовании байесовского подхода следующая (см. также рис. ниже).  
Выбирается набор возможных моделей $model$ (или форма модели и область параметров для нее).  
Для каждого набора параметров задается априорная вероятность $P(model)$.  
Вычисляется функция правдоподобия $P(data|model)$ - вероятность получить данные в рамках выбранной модели.    
Вычисляется апостериорная вероятность $P(model|data)$.  
Анализируются свойства моделей и постериорных распределений.  

# Конверсии

Функция правдоподобия задается биномиальным распределением.  

Априорное распределение конверсий удобно задать бета-распределением. Без учета нормировочных коэффициентов зависимость от $p$ $\mbox{Beta}(p; \alpha, \beta) \propto p^{\alpha-1}(1-p)^{\beta-1}$. Эта зависимость сохранится для произведения правдоподобия на априорное распределение. Априорные распределения с таким свойством называют сопряженными априорными распределениями.  

При $\alpha=1, \beta=1$ бета-распределение совпадает с однородным - можно выбирать эти значения как стартовые.   
Либо можно подобрать на основе ожидаемого эффекта.

$$
P(model | data) \propto P(data | model) P(model)
$$

$$
P(data | model) = P(n_s | p, N) \sim \mbox{Binom}(p, N) = C_{N}^{n_s} p^{n_s} (1-p)^{N-n_s}
$$

$$
P(model) = P(p) \sim \mbox{Beta}(p; \alpha, \beta) = 
\frac{\Gamma(\alpha + \beta)}{\Gamma(\alpha) \Gamma(\beta)} p^{\alpha-1}(1-p)^{\beta-1}
$$

$$
\begin{split}
P(model | data) & = P(p | n_s)
\\
& \propto \mbox{Binom}(p, N) \mbox{Beta}(p; \alpha, \beta)
\\
& \propto C_{N}^{n_s} p^{n_s} (1-p)^{N-n_s}
\frac{\Gamma(\alpha + \beta)}{\Gamma(\alpha) \Gamma(\beta)} p^{\alpha-1}(1-p)^{\beta-1}
\\
& \propto p^{n_s + \alpha - 1} (1-p)^{N - n_s + \beta - 1}
\\
& = \mbox{Beta}(p; \alpha + n_s, \beta + N - n_s)
\end{split}
$$

Примеры бета-распределения на графике ниже [[BetaDist](https://en.wikipedia.org/wiki/Beta_distribution), [SciPyBeta](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.beta.html)].
При $\alpha = 1, \beta=1$ распределение однородное.  
Максимум в точке $p = (\alpha-1) / (\alpha + \beta - 2)$. При увеличении $\alpha$ и $\beta$ распределение сужается.  
В пределе совпадает с нормальным.  

In [ ]:
x = np.linspace(0, 1, 1000)
fig = go.Figure()
a, b = 1, 1
fig.add_trace(go.Scatter(x=x, y=stats.beta.pdf(x, a=a, b=b), 
                             mode='lines', line_color='black', line_dash='dash',
                             name=f'a={a}, b={b}'))
a, b = 1, 5
fig.add_trace(go.Scatter(x=x, y=stats.beta.pdf(x, a=a, b=b), 
                             mode='lines', line_color='black', line_dash='solid',
                             name=f'a={a}, b={b}'))
a, b = 3, 5
fig.add_trace(go.Scatter(x=x, y=stats.beta.pdf(x, a=a, b=b), 
                             mode='lines', line_color='black', line_dash='solid',
                             name=f'a={a}, b={b}'))
a, b = 25, 30
fig.add_trace(go.Scatter(x=x, y=stats.beta.pdf(x, a=a, b=b), 
                             mode='lines', line_color='black', line_dash='solid',
                             name=f'a={a}, b={b}'))
a, b = 150, 50
fig.add_trace(go.Scatter(x=x, y=stats.beta.pdf(x, a=a, b=b), 
                             mode='lines', line_color='black', line_dash='solid',
                             name=f'a={a}, b={b}')) 
fig.add_trace(go.Scatter(
    x=[0.98, 0.08, 0.32, 0.55, 0.87],
    y=[1.35, 5.00, 2.80, 6.20, 12.0],
    mode="text",
    name=None,
    showlegend=False,
    text=["a=1, b=1", "a=1, b=5", "a=3, b=5", "a=25, b=30", "a=150, b=50"],
    textposition="top left"
))
fig.update_layout(title='Бета-распределение Beta(a, b)',
                  xaxis_title='x',
                  yaxis_title='Плотность вероятности',
                  showlegend=False,
                  xaxis_range=[0, 1],
                  height=550)
fig.show()

Оценка параметров

In [ ]:
def posterior_dist_binom(ns, ntotal, a_prior=1, b_prior=1):
    a = a_prior + ns
    b = b_prior + (ntotal - ns) 
    return stats.beta(a=a, b=b)
    
def posterior_binom_approx_95pdi(post_dist):
    lower = post_dist.ppf(0.025)
    upper = post_dist.ppf(0.975)
    return lower, upper

def prob_pb_gt_pa(post_dist_A, post_dist_B, post_samp=100_000):
    sa = post_dist_A.rvs(size=post_samp)
    sb = post_dist_B.rvs(size=post_samp)
    b_gt_a = np.sum(sb > sa)
    return b_gt_a / post_samp

p = 0.1
nsample = 1000

exact_dist = stats.bernoulli(p=p)
data = exact_dist.rvs(nsample)

post_dist = posterior_dist_binom(ns=np.sum(data), ntotal=len(data))

x = np.linspace(0, 1, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=post_dist.pdf(x), line_color='black', name='Апостериорное'))
fig.add_trace(go.Scatter(x=[np.sum(data)/len(data), np.sum(data)/len(data)], y=[0, max(post_dist.pdf(x))], 
                         line_color='black', mode='lines', line_dash='dash', name='Среднее в выборке'))
fig.add_trace(go.Scatter(x=[exact_dist.mean(), exact_dist.mean()], y=[0, max(post_dist.pdf(x))*1.05], 
                         line_color='red', mode='lines', line_dash='dash', name='Точное p'))
fig.update_layout(title='Апостериорное распределение',
                  xaxis_title='p',
                  yaxis_title='Плотность вероятности',
                  xaxis_range=[0, 1],
                  hovermode="x",
                  height=500)
fig.show()

2 группы

In [ ]:
p_A = 0.1
p_B = p_A * 1.05
nsample = 1000

exact_dist_A = stats.bernoulli(p=p_A)
exact_dist_B = stats.bernoulli(p=p_B)
data_A = exact_dist_A.rvs(nsample)
data_B = exact_dist_B.rvs(nsample)

post_dist_A = posterior_dist_binom(ns=np.sum(data_A), ntotal=len(data_A))
post_dist_B = posterior_dist_binom(ns=np.sum(data_B), ntotal=len(data_B))

x = np.linspace(0, 1, 1000)
fig = go.Figure()
#fig.add_vline(x=exact_dist.mean(), line_dash='dash', name='Exact')
fig.add_trace(go.Scatter(x=x, y=post_dist_A.pdf(x), line_color='black', name='A'))
fig.add_trace(go.Scatter(x=x, y=post_dist_B.pdf(x), line_color='black', opacity=0.2, name='B'))
fig.add_trace(go.Scatter(x=[exact_dist_A.mean(), exact_dist_A.mean()], y=[0, max(post_dist_A.pdf(x))*1.05], 
                         mode='lines', line_dash='dash', line_color='black', name='Точное A'))
fig.add_trace(go.Scatter(x=[exact_dist_B.mean(), exact_dist_B.mean()], y=[0, max(post_dist_A.pdf(x))*1.05], 
                         mode='lines', line_dash='dash', line_color='black', opacity=0.2, name='Точное B'))
fig.update_layout(title='Апостериорные распределения',
                  xaxis_title='p',
                  yaxis_title='Плотность вероятности',
                  xaxis_range=[p_A/2, p_A*2],
                  hovermode="x",
                  height=500)
fig.show()

print(f'P(pB > pA): {prob_pb_gt_pa(post_dist_A, post_dist_B)}')

Динамика по мере набора данных

Количество правильно угаданных вариантов

In [ ]:
cmp = pd.DataFrame(columns=['A', 'B', 'best_exact', 'exp_samp_size', 'A_exp', 'B_exp', 'best_exp', 'p_best'])

p = 0.1
nexps = 100
cmp['A'] = [p] * nexps
cmp['B'] = p * (1 + stats.uniform.rvs(loc=-0.05, scale=0.1, size=nexps))
cmp['best_exact'] = cmp.apply(lambda r: 'B' if r['B'] > r['A'] else 'A', axis=1)

prob_stop = 0.95
for i in range(nexps):
    pA = cmp.at[i, 'A']
    pB = cmp.at[i, 'B']
    exact_dist_A = stats.bernoulli(p=pA)
    exact_dist_B = stats.bernoulli(p=pB)
    n_samp_max = 10_000_000
    n_samp_total = 0
    n_samp_step = 10_000
    ns_A = 0
    ns_B = 0
    while n_samp_total < n_samp_max:
        dA = exact_dist_A.rvs(n_samp_step)
        dB = exact_dist_B.rvs(n_samp_step)
        n_samp_total += n_samp_step
        ns_A = ns_A + np.sum(dA)
        ns_B = ns_B + np.sum(dB)
        post_dist_A = posterior_dist_binom(ns=ns_A, ntotal=n_samp_total)
        post_dist_B = posterior_dist_binom(ns=ns_B, ntotal=n_samp_total)
        pb_gt_pa = prob_pb_gt_pa(post_dist_A, post_dist_B)
        best_gr = 'B' if pb_gt_pa >= prob_stop else 'A' if (1 - pb_gt_pa) >= prob_stop else None
        if best_gr:
            cmp.at[i, 'A_exp'] = post_dist_A.mean()
            cmp.at[i, 'B_exp'] = post_dist_B.mean()
            cmp.at[i, 'exp_samp_size'] = n_samp_total
            cmp.at[i, 'best_exp'] = best_gr
            cmp.at[i, 'p_best'] = pb_gt_pa
            break
    print(f'done {i}: {n_samp_total}, {best_gr}, {pb_gt_pa}')

cmp['correct'] = cmp['best_exact'] == cmp['best_exp']
display(cmp.head(10))
cor_guess = np.sum(cmp['correct'])
print(f"Nexp: {nexps}, Correct Guesses: {cor_guess}, Accuracy: {cor_guess / nexps}")

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=cmp['exp_samp_size'], histnorm='probability', name='Groups Sample Size', 
                           marker_color='black', nbinsx=100))
fig.update_layout(title='Размер выборки',
                  xaxis_title='Точек в группе',
                  yaxis_title='Доля экспериментов',
                  hovermode="x",
                  height=500)
fig.show()

# Средние

Байесовский подход [[SGBS](https://www.amazon.co.uk/Students-Guide-Bayesian-Statistics/dp/1473916364), [SR](https://www.amazon.co.uk/Statistical-Rethinking-Bayesian-Examples-Chapman/dp/036713991X/ref=sr_1_1)] требует построения моделей распределений сравниваемых величин. Выбор модели - неочевидный вопрос. Есть отдельные модели, получившие распространение - например, Buy Till You Die [[BTYD](https://en.wikipedia.org/wiki/Buy_Till_you_Die)] для количества покупок клиента за все время использования сервиса. Но универсальных моделей нет. Это создает сложности.

При этом для многих величин не всегда нужно знать все распределение. Можно ограничиться сравнением средних:
средней выручкой на пользователя, средней длительностью просмотра и т.д. Для средних значений часто применима центральная предельная теорема. Можно приближенно считать, что средние значения в выборках из распределения будут распределены нормально. Причем это не зависит от формы исходного распределения. Центральная предельная теорема позволяет при сравнении средних использовать нормальное распределение в качестве функции правдоподобия.

Есть несколько центральных предельных теорем [[CLT](https://en.wikipedia.org/wiki/Central_limit_theorem)].
Одна из возможных формулировок следующая. Пусть есть последовательность независимых одинаково распределенных случайных величин $X_1, X_2, \dots, X_n, \dots$ с конечными математическим ожиданием $\mu$ и дисперсией $\sigma^2$. Пусть $\bar{X}_n = \frac{1}{n} \sum_{i=1}^{n} X_i$ их выборочное среднее. Тогда при $n$, стремящемся к бесконечности, распределение центрированных и масштабированных выборочных средних сходится к нормальному распределению [[NormDist](https://en.wikipedia.org/wiki/Normal_distribution), [SciPyNorm](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html?highlight=norm)] со средним значением 0 и дисперсией 1

$$
P \left( \frac{\bar{X}_n - \mu}{\sigma / \sqrt{n}} = x \right) \to N(x; 0, 1), \quad n \to \infty,
\\
N(x ; \mu, \sigma^2) = \frac{1}{\sqrt{2 \pi \sigma^2}} e^{- \frac{(x-\mu)^2}{2 \sigma^2} } .
$$

Сходимость понимается как сходимость по распределению [[RandVarsConv](https://en.wikipedia.org/wiki/Convergence_of_random_variables#Convergence_in_distribution)].

Неформально этот результат можно применить следующим образом. Если взять произвольное распределение со средним значением $\mu$ и диспресий $\sigma^2$, начать выбирать из него сэмплы длины $n$ и считать среднее в каждом сэмпле, то средние значения сэмплов будут распределены приблизительно нормально $N(\mu, \sigma^2/n)$.

<center>
<img src="./figs/central_limit_theorem.png" alt="central_limit_theorem" width="600"/>
</center>

Центральная предельная теорема говорит о сходимости к нормальному распределению центрированных и масштабированных выборочных средних $\bar{X}_n$ при стремлении $n$ к бесконечности. Для фиксированного конечного числа $n$ нормальное распределение не гарантируется. При этом есть теоремы, дающие оценку отличия распределения суммы конечного количества случайных величин от нормального - см. [[BerryEsseenTheorem](https://en.wikipedia.org/wiki/Berry%E2%80%93Esseen_theorem)]. Отличие зависит как от количества слагаемых, так и от параметров распределения. 

In [ ]:
a = 1
sample_len = 100
n_samples = 1000

exact_dist = stats.gamma(a=a)
samp = exact_dist.rvs(size=(n_samples, sample_len))
means = np.array([x.mean() for x in samp])
clt_mu = exact_dist.mean()
clt_stdev = exact_dist.std() / np.sqrt(sample_len)
means_stdev = means.std()

x = np.linspace(0, 10, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), 
                         mode='lines', line_color='black', line_dash='dash', name='Исходное распределение'))
fig.add_trace(go.Histogram(x=np.concatenate(samp), histnorm='probability density', name='Выборка', nbinsx=500,
                           marker_color='black', opacity=0.3))
#fig.add_vline(exact_dist.mean(), name='Точное среднее')
fig.add_trace(go.Scatter(x=x, y=stats.norm.pdf(x, loc=clt_mu, scale=clt_stdev), 
                         mode='lines', line_color='black', line_dash='solid', name='ЦПТ-распределение'))
fig.add_trace(go.Histogram(x=means, histnorm='probability density', name='Выборочные средние', nbinsx=50,
                           marker_color='green', opacity=0.5))
fig.update_layout(title='Выборочные средние',
                  xaxis_title='x',
                  yaxis_title='Плотность вероятности',
                  barmode='overlay',
                  hovermode="x",
                  height=550)
fig.update_layout(xaxis_range=[0, 5])
fig.show()

В приведенной формулировке центральная предельная теорема требует существования конечных среднего и дисперсии у исходного распределения. Примерами распределений, для которых эти свойства могут не выполнятся, являются распределение Парето [[ParetoDist](https://en.wikipedia.org/wiki/Pareto_distribution), [SciPyPareto](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pareto.html)] и близкое к нему распределение Ломакса [[LomaxDist](https://en.wikipedia.org/wiki/Lomax_distribution), [SciPyLomax](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.lomax.html)]. Плотность вероятности последнего имеет вид

$$
P(x; c) = \frac{c}{(1 + x )^{c + 1}}, \quad x \ge 0, c > 0.
$$

При значениях параметра $c$ меньше или равном 2 дисперсия распределения Ломакса не является конечной. 
На графиках ниже приведена гистограмма средних в сэмплах и нормальное распределение с параметрами, равными среднему и дисперсии средних сэмплов.

In [ ]:
c = 1.7
sample_len = 500
n_samples = 1000

exact_dist = stats.lomax(c=c)
samp = exact_dist.rvs(size=(n_samples, sample_len))
means = np.array([x.mean() for x in samp])
clt_mu = exact_dist.mean()
clt_stdev = exact_dist.std() / np.sqrt(sample_len)
means_stdev = means.std()

xaxis_max=10
x = np.linspace(0, xaxis_max, 2000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), 
                         mode='lines', line_color='black', line_dash='dash', name='Исходное распределение'))
fig.add_trace(go.Histogram(x=np.concatenate(samp)[np.concatenate(samp) < xaxis_max], histnorm='probability density', 
                           name='Выборка', nbinsx=500,
                           marker_color='black', opacity=0.3))
#fig.add_vline(exact_dist.mean(), name='Точное среднее')
fig.add_trace(go.Scatter(x=x, y=stats.norm.pdf(x, loc=clt_mu, scale=means_stdev), 
                         mode='lines', line_color='black', line_dash='solid', name='ЦПТ-подобное распределение'))
fig.add_trace(go.Histogram(x=means, histnorm='probability density', name='Выборочные средние', nbinsx=150,
                          marker_color='green', opacity=0.5))
fig.update_layout(title='Выборочные средние',
                  xaxis_title='x',
                  yaxis_title='Плотность вероятности',
                  barmode='overlay',
                  hovermode="x",
                  xaxis_range=[0, xaxis_max],
                  height=550)
fig.show()

# Выручка на пользователя

# Заказы на платящего

Заказы пользователя дискретная величина $P_{заказы}(n)$, $n \in 1, 2, \dots$. Можно ожидать лог-нормальное или степенное распределение - Парето, распределение Ципфа [[ZipfDist](https://en.wikipedia.org/wiki/Zipf%27s_law#Formal_definition)] $P(n) \propto n^{-s}$. Модель с меньшими ограничениями - свои вероятности $p_i$ под каждое количество заказов. Пусть масимальное количество заказов $N$ ограничено, $n_i$ - количество пользователей с $i=1, 2, \dots, N$ заказами, $p_i$ - вероятность сделать $i$ заказов. Функция правдоподобия задается мультиномиальным распределением [[MultiDist](https://en.wikipedia.org/wiki/Multinomial_distribution)]

$$
P(data|model) = P(n_1, \dots, n_N) = \frac{(n_1 + \dots + n_N)!}{n_{1}! \dots n_{N}!} p_{1}^{n_{1}} \dots p_{N}^{n_{N}} .
$$

Распределение Дирихле [[DrDist](https://en.wikipedia.org/wiki/Dirichlet_distribution)] будет сопряженным априорным распределением

$$
P(model) = 
Dir \left( p_{1}, \dots, p_{N}; \alpha_{1}, \dots, \alpha_{N} \right) = 
\dfrac{1}{B( \alpha_{1}, \dots, \alpha_{N} )} \prod_{i=1}^{N} p_{i}^{\alpha_{i}-1},
\qquad
\sum_{i=1}^{N} p_i = 1,
\qquad
p_i \in [0, 1], 
\qquad
B(\alpha_{1}, \dots, \alpha_{N}) = 
\frac{\prod \limits_{i=1}^{N} \Gamma( \alpha_{i} )}
{\Gamma \left( \sum \limits_{i=1}^{N} \alpha_{i} \right)} .
$$

Апостериорное распределение 

$$
\begin{split}
P(model | data) 
& \propto
\frac{(n_1 + \dots + n_N)!}{n_{1}! \dots n_{N}!} p_{1}^{n_{1}} \dots p_{N}^{n_{N}}
\dfrac{1}{B(\alpha_{1}, \dots ,\alpha_{N})} \prod _{i=1}^{N} p_{i}^{\alpha_{i}-1}
\\
& \propto
\prod_{i=1}^{N} p_{i}^{n_{i} + \alpha_{i} - 1}
\\
& =
Dir \left( p_{1}, \dots, p_{N}; n_1 + \alpha_{1}, \dots, n_N + \alpha_{N} \right)
\end{split}
$$

Маржинальными распределениями для каждого $p_i$ будут бета-распределения

$$
\begin{split}
f(p_i) = 
Beta( p_i; \alpha_i, \alpha_0 - \alpha_i ),
\quad
\alpha_0 = \sum_{i=1}^{N} \alpha_i .
\end{split}
$$

Оценка параметров

In [ ]:
def initial_params_dr(N):
    return np.ones(N)

def posterior_params_dr(data, initial_pars):
    u, c = np.unique(data, return_counts=True)
    post_pars = np.copy(initial_pars)
    for k, v in zip(u, c):
        post_pars[k - 1] = post_pars[k - 1] + v
    return post_pars

def posterior_nords_rvs(params, nsamp):
    nords = np.empty(nsamp)
    probs = stats.dirichlet.rvs(alpha=params, size=nsamp)
    for i, p in enumerate(probs):
        nords[i] = np.argmax(stats.multinomial.rvs(n=1, p=p)) + 1
    return nords

def posterior_marginal_dist_pi(i, params):
    imin = 1
    p_i = stats.beta(a=params[i - imin], b=np.sum(params) - params[i - imin])
    return p_i

def posterior_pi_mean_95pdi(i, params):
    p = posterior_marginal_dist_pi(i, params)
    m = p.mean()
    lower = p.ppf(0.025)
    upper = p.ppf(0.975)
    return m, lower, upper

def posterior_nords_mean(params):
    ex = 0
    for i in range(1, len(params)+1):
        m = posterior_marginal_dist_pi(i, params).mean()
        ex += i * m
    return ex

Nmax = 30
s = 1.5
nsample = 1000

exact_dist = stats.zipfian(a=s, n=Nmax)
data = exact_dist.rvs(nsample)

pars = initial_params_dr(Nmax)
pars = posterior_params_dr(data, pars)

post_samp = posterior_nords_rvs(pars, 100000)

pi = [posterior_pi_mean_95pdi(i, pars) for i in range(1, Nmax+1)]

x = np.arange(1, Nmax+1)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pmf(x), name='Точное распределение Ципфа'))
fig.add_trace(go.Histogram(x=data, histnorm='probability', name='Сэмпл', nbinsx=round(Nmax*2)))
fig.add_trace(go.Histogram(x=post_samp, histnorm='probability', name='Апостериорное распределение', opacity=0.5, nbinsx=round(Nmax*2)))
fig.add_trace(go.Scatter(x=x, 
                         y=[p[0] for p in pi],
                         error_y=dict(type='data', symmetric=False, array=[p[2] - p[0] for p in pi], arrayminus=[p[0] - p[1] for p in pi]), 
                         name='$\mbox{Оценки } p_i$',
                         mode='markers'))
fig.update_layout(title='Заказы на платящего',
                  xaxis_title='$x$',
                  yaxis_title='Вероятность',
                  xaxis_range=[0, Nmax+1],
                  hovermode="x",
                  barmode="group",
                  height=550)
fig.show()

Сравнение двух групп

In [ ]:
def prob_pb_gt_pa_postsamp(post_samp_a, post_samp_b):
    if len(post_samp_a) != len(post_samp_b):
        return None
    b_gt_a = np.sum(post_samp_b > post_samp_a)
    return b_gt_a / len(post_samp_a)

Nmax = 30
s_a = 2.7
s_b = s_a * 1.03
nsample = 3000

exact_dist_a = stats.zipfian(a=s_a, n=Nmax)
exact_dist_b = stats.zipfian(a=s_b, n=Nmax)
data_a = exact_dist_a.rvs(nsample)
data_b = exact_dist_b.rvs(nsample)

pars_a = initial_params_dr(Nmax)
pars_a = posterior_params_dr(data_a, pars_a)
pars_b = initial_params_dr(Nmax)
pars_b = posterior_params_dr(data_b, pars_b)

post_samp_len = 100000
post_samp_a = posterior_nords_rvs(pars_a, post_samp_len)
post_samp_b = posterior_nords_rvs(pars_b, post_samp_len)

pi_a = [posterior_pi_mean_95pdi(i, pars) for i in range(1, Nmax+1)]

x = np.arange(1, Nmax+1)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist_a.pmf(x), name='Точное распределение A'))
fig.add_trace(go.Scatter(x=x, y=exact_dist_b.pmf(x), name='Точное распределение A'))
fig.add_vline(x=exact_dist_a.mean())
fig.add_vline(x=exact_dist_b.mean())
fig.add_trace(go.Histogram(x=post_samp_a, histnorm='probability', name='Апостериорное A', opacity=0.5, nbinsx=round(Nmax*2)))
fig.add_trace(go.Histogram(x=post_samp_b, histnorm='probability', name='Апостериорное B', opacity=0.5, nbinsx=round(Nmax*2)))
# fig.add_trace(go.Scatter(x=x, 
#                          y=[p[0] for p in pi],
#                          error_y=dict(type='data', symmetric=False, array=[p[2] - p[0] for p in pi], arrayminus=[p[0] - p[1] for p in pi]), 
#                          name='$\mbox{Оценки } p_i$',
#                          mode='markers'))
fig.update_layout(title='Заказы на платящего',
                  xaxis_title='$x$',
                  yaxis_title='Вероятность',
                  xaxis_range=[0, Nmax+1],
                  hovermode="x",
                  barmode="group",
                  height=550)
fig.show()

print(f'E[n_ords] A:, {posterior_nords_mean(pars_a)}, B: {posterior_nords_mean(pars_b)}')
print(f'P(pB > pA): {prob_pb_gt_pa_postsamp(post_samp_a, post_samp_b)}')

Количество правильно угаданных вариантов

In [ ]:
cmp = pd.DataFrame(columns=['A', 'B', 'best_exact', 'exp_samp_size', 'A_exp', 'B_exp', 'best_exp', 'p_best'])

s = 2.7
Nmax=30
nexps = 100
cmp['A'] = [s] * nexps
cmp['B'] = s * (1 + stats.uniform.rvs(loc=-0.05, scale=0.1, size=nexps))
cmp['best_exact'] = cmp.apply(lambda r: 'B' if r['B'] < r['A'] else 'A', axis=1)

prob_stop = 0.90
for i in range(nexps):
    s_a = cmp.at[i, 'A']
    s_b = cmp.at[i, 'B']
    exact_dist_a = stats.zipfian(a=s_a, n=Nmax)
    exact_dist_b = stats.zipfian(a=s_b, n=Nmax)
    n_samp_max = 200000
    n_samp_total = 0
    n_samp_step = 10000
    pars_a = initial_params_dr(Nmax)
    pars_b = initial_params_dr(Nmax)
    while n_samp_total < n_samp_max:
        data_a = exact_dist_a.rvs(n_samp_step)
        data_b = exact_dist_b.rvs(n_samp_step)
        n_samp_total += n_samp_step
        pars_a = posterior_params_dr(data_a, pars_a)
        pars_b = posterior_params_dr(data_b, pars_b)
        post_samp_len = 10000
        post_samp_a = posterior_nords_rvs(pars_a, post_samp_len)
        post_samp_b = posterior_nords_rvs(pars_b, post_samp_len)
        pb_gt_pa = prob_pb_gt_pa_postsamp(post_samp_a, post_samp_b)
        best_gr = 'B' if pb_gt_pa >= prob_stop else 'A' if (1 - pb_gt_pa) >= prob_stop else None
        if best_gr:
            cmp.at[i, 'A_exp'] = post_dist_A.mean()
            cmp.at[i, 'B_exp'] = post_dist_B.mean()
            cmp.at[i, 'exp_samp_size'] = n_samp_total
            cmp.at[i, 'best_exp'] = best_gr
            cmp.at[i, 'p_best'] = pb_gt_pa
            break
    print(f'done {i}: {n_samp_total}, {best_gr}, {pb_gt_pa}')

cmp['correct'] = cmp['best_exact'] == cmp['best_exp']
display(cmp.head(10))
cor_guess = np.sum(cmp['correct'])
print(f"Nexp: {nexps}, Correct Guesses: {cor_guess}, Accuracy: {cor_guess / nexps}")

# Средний чек

# Заключение

# Ссылки

[BinomDist] - [Binomial Distribution](https://en.wikipedia.org/wiki/Binomial_distribution), *Wikipedia.*  
[SciPyBinom] - [scipy.stats.binom](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.binom.html), *SciPy Reference.*   
[BetaDist] - [Beta Distribution](https://en.wikipedia.org/wiki/Beta_distribution), *Wikipedia.*     
[SciPyBeta] - [scipy.stats.beta](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.beta.html), *SciPy Reference.*   

Пациент приходит ко врачу. Симптомы - температура.   
Врач делает предположения на счет болезней - простуда, пневмония и т.д.  
Врач прикидывает, при каких болезнях эти симптомы вероятны.  
Дальше с учетом своего опыта и распространенности болезней ставит диагноз.  

Можно записать рассуждения количественно.
Для диагноза интересует $P(болезнь | симптомы)$.  
Вначале выбирается список возможных болезней и для каждой вычисляется $P(симптомы | болезнь)$.  
Распространенность болезней в регионе и опыт врача учитывает $P(болезнь)$.  

$$
P(болезнь | симптомы) \approx P(симптомы | болезнь) P(болезнь)
$$

На этом примере видны компоненты байесовского моделирования.  

Let’s walk through an example of using the **Bayesian approach** to guess what’s inside a present based on the **size of the box**.

### Setting:
You receive a box as a gift, but you don’t know what’s inside. Based on the **size of the box**, you want to update your belief about what the present might be. You have a few reasonable guesses, and you’ll use **Bayes’ Theorem** to compute the posterior probabilities of each possible item given the size of the box.

### Hypotheses (Possible Contents of the Box):
You consider three possible items that could be inside the box:

- \( H_1 \): A **book**.
- \( H_2 \): A **pair of shoes**.
- \( H_3 \): A **laptop**.

### Prior Probabilities:
Before looking at the size of the box, you have some prior beliefs about what the gift could be based on your knowledge of the person who gave you the gift. For example:

- \( P(H_1) = 0.3 \) (They often give books as gifts).
- \( P(H_2) = 0.5 \) (They’ve given shoes before).
- \( P(H_3) = 0.2 \) (They once mentioned buying you a laptop).

### Likelihoods (Based on the Size of the Box):
Now, you observe the **size of the box**, and you know how likely it is for each possible item to fit into a box of this size. Suppose the box is medium-sized. Based on that, you assign the following likelihoods:

- \( P(\text{Medium box} \mid H_1) = 0.8 \) (Books often come in medium-sized boxes).
- \( P(\text{Medium box} \mid H_2) = 0.6 \) (Shoes could fit in a medium-sized box, but not always).
- \( P(\text{Medium box} \mid H_3) = 0.3 \) (Laptops usually come in larger boxes, so this is less likely).

### Evidence (Total Probability of Observing the Box Size):
The total probability of receiving a **medium-sized box** is calculated by summing over all the possible contents:
\[
P(\text{Medium box}) = P(\text{Medium box} \mid H_1)P(H_1) + P(\text{Medium box} \mid H_2)P(H_2) + P(\text{Medium box} \mid H_3)P(H_3)
\]
\[
P(\text{Medium box}) = (0.8 \times 0.3) + (0.6 \times 0.5) + (0.3 \times 0.2) = 0.24 + 0.3 + 0.06 = 0.6
\]

### Applying Bayes’ Theorem:
Now, you can compute the **posterior probability** for each possible item using **Bayes’ Theorem**:

1. **Posterior for a Book**:
   \[
   P(H_1 \mid \text{Medium box}) = \frac{P(\text{Medium box} \mid H_1) P(H_1)}{P(\text{Medium box})} = \frac{0.8 \times 0.3}{0.6} = \frac{0.24}{0.6} = 0.4
   \]

2. **Posterior for Shoes**:
   \[
   P(H_2 \mid \text{Medium box}) = \frac{P(\text{Medium box} \mid H_2) P(H_2)}{P(\text{Medium box})} = \frac{0.6 \times 0.5}{0.6} = \frac{0.3}{0.6} = 0.5
   \]

3. **Posterior for a Laptop**:
   \[
   P(H_3 \mid \text{Medium box}) = \frac{P(\text{Medium box} \mid H_3) P(H_3)}{P(\text{Medium box})} = \frac{0.3 \times 0.2}{0.6} = \frac{0.06}{0.6} = 0.1
   \]

### Conclusion:
Based on the size of the box and your priors, you now have updated probabilities for what might be inside the box:
- \( P(\text{Book} \mid \text{Medium box}) = 0.4 \),
- \( P(\text{Shoes} \mid \text{Medium box}) = 0.5 \),
- \( P(\text{Laptop} \mid \text{Medium box}) = 0.1 \).

The most likely item is a **pair of shoes** (50% chance), followed by a **book** (40% chance). A **laptop** is the least likely (10% chance).

Аномалия в дашборде. Как выглядит процесс расследования причин?  
Вначале нужно сформулировать возможные предположения: баг в недавнем релизе, не отработали ETL-скрипты, ... .
С учетом прошлого опыта выбрать наиболее вероятное.  
Проверить.  

Можно записать рассуждения количественно.
Интересует $P(изменения | аномалия)$.  
Вначале выбирается список возможных причин и для каждой вычисляется $P(аномалия | изменения)$.  
Прошлый опыт учитывает $P(изменения)$.  

$$
P(изменения | аномалия) \approx P(аномалия | изменения) P(изменения)
$$

На этом примере видны компоненты байесовского моделирования. 

* В мешке лежат черные и белые шары, всего 1000 штук. Для оценки количества черных шаров вынимают шар, записывают цвет, возвращают обратно. За 20 повторов вынули 5 черных шаров. Сколько может быть черных шаров в мешке и каковы вероятности этих значений?

1)

При количестве черных шаров $N_B$ вероятность достать черный шар $p_B = N_B / N$.  
Вероятность вынуть 5 черных шаров в 20 экспериментах задается биномиальным распределением.

$$
P(5, 20 | N_B) = Binom(5, 20) = C^5_{20} p_B^5 p_W^{15}
$$

Возможные значения $N_B$ от 1 (мог попадаться один и тот же черный шар) до 999 (есть как минимум 1 белый).

Интересует

$$
P(N_B | 5, 20) = \frac{ P(5,20 | N_B) P(N_B)}{P(5,20)} = \frac{ P(5,20 | N_B) P(N_B)}{\sum \limits_{N_B=1}^{999} P(5,20 | N_B) P(N_B)}
$$

В знаменателе нормировочный коэффициент.

Значения $P(N_B)$ равновероятны $P(N_B) = 1/999$.

$$
P(N_B | 5, 20) 
= \frac{ P(5,20 | N_B)}{\sum \limits_{N_B=1}^{999} P(5,20 | N_B)} 
= \frac{C^5_{20} p_B^5 p_W^{15}}{\sum \limits_{N_B=1}^{999} C^5_{20} p_B^5 p_W^{15}}
= \frac{C^5_{20} N_B^5 (N - N_B)^{15}}{\sum \limits_{N_B=1}^{999} C^5_{20} N_B^5 (N-N_B)^{15}}
$$

Лучше так:
1000 шаров, достали-вернули 5, все 5 черные. Сколько всего черных?


$$
P(5 | N_B) = Binom(5, 5) = C^5_{5} p_B^5 p_W^{0} = p_B^5
$$

$$
P(N_B | 5) = \frac{ P(5 | N_B) P(N_B)}{\sum \limits_{N_B=1}^{1000} P(5 | N_B) P(N_B)}
$$

Значения $P(N_B)$ равновероятны $P(N_B) = 1/1000$.

$$
P(N_B | 5) 
= \frac{ P(5 | N_B)}{\sum \limits_{N_B=1}^{1000} P(5 | N_B)} 
= \frac{p_B^5}{\sum \limits_{N_B=1}^{1000} p_B^5}
= \frac{N_B^5}{\sum \limits_{N_B=1}^{1000} N_B^5}
$$

$$
\mbox{Beta}(1, 1) = 
\mbox{Uniform}(p) = 
\begin{cases}
1, p \in [0, 1]
\\
0, p \not\in [0, 1]
\end{cases}
$$

In [ ]:
d = np.arange(0, 30)

p = 0.05
N = 5000
ns = stats.binom.rvs(n=N, p=p, size=len(d))
cr = ns / N
p_drop = p*2/3
ns_drop = stats.binom.rvs(n=N, p=p_drop, size=len(d)//2)
cr_drop = ns_drop / N
p_inc = p*1.03
ns_inc = stats.binom.rvs(n=N, p=p_inc, size=len(d)//2)
cr_inc = ns_inc / N

fig = go.Figure()
fig.add_trace(go.Scatter(x=d, y=cr*100, 
                        name='Базовое значение', line_color='black'))
fig.add_trace(go.Scatter(x=d[len(d)//2-1:], y=np.concatenate([cr[[len(d)//2-1]]*100, cr_drop*100]),
                         name='-30%', line_color='black', line_dash='dash'))
fig.add_trace(go.Scatter(x=d[len(d)//2-1:], y=np.concatenate([cr[[len(d)//2-1]]*100, cr_inc*100]),
                        name='+3%', line_color='black', opacity=0.4, line_dash='dash'))
fig.update_layout(title='Эффект',
                  xaxis_title='Дни',
                  yaxis_title='Метрика',
                  xaxis_range=[0, len(d)-1],
                  xaxis_showticklabels=False,
                  yaxis_range=[1, 7],
                  yaxis_showticklabels=False,
                  hovermode="x",
                  barmode="group",
                  width=900)
fig.show()

In [ ]:
fig.write_image("./effect_size.png", scale=2)